In [11]:
import os
import pandas as pd

In [12]:
exp_root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged"

datasets = {
    "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

In [13]:
df_index = pd.read_csv(os.path.join(exp_root, "exp_status.csv"))
df_index

,model,task,dataset,model_type,trainable_module,path,have_eval_result,have_prediction,have_gpt_score,model_family
0,Qwen2-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen2-VL/eval_seed0/Qwen2-VL-7B-Inst...,1,1,1,Qwen
1,Qwen25-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen25-VL/eval_seed0/Qwen2.5-VL-7B-I...,1,1,1,Qwen
2,Gemma3,vqa,SLAKE,general,NaN,vqa/SLAKE/Gemma3/eval_seed0/gemma-3-4b-it,1,1,1,Gemma
3,MedGemma,vqa,SLAKE,medical,NaN,vqa/SLAKE/MedGemma/eval_seed0/medgemma-4b-it,1,1,1,Gemma
4,InternVL3,vqa,SLAKE,general,NaN,vqa/SLAKE/InternVL3/eval_seed0/InternVL3-8B-hf,1,1,1,Intern
...,...,...,...,...,...,...,...,...,...,...
83,VILA,vqa,VQA-RAD,general,ML,vqa/VQA-RAD/VILA1.5/eval_seed0/1epoch-lora8,1,1,1,VILA
84,VILA-M3,vqa,VQA-RAD,medical,ML,vqa/VQA-RAD/VILA-M3/eval_seed0/1epoch-lora8,1,1,1,VILA
85,Lingshu,vqa,Harvard-FairVLMed10k,medical,ML,vqa/Harvard-FairVLMed10k/Lingshu/eval_seed0/1e...,1,1,1,Qwen
86,VILA,vqa,Harvard-FairVLMed10k,general,ML,vqa/Harvard-FairVLMed10k/VILA1.5/eval_seed0/1e...,1,1,1,VILA


In [14]:
open_metrics = [
    "bleu1",
    "bleu2",
    "bleu3",
    "bleu4",
    "rouge1",
    "rouge2",
    "rougeL",
    "exact_match",
    "recall",
    "precision",
    "f1_score",
    "accuracy",
    # "bertscore",
    # "meteor",
    "gpt_score",
]


closed_metrics = [
    "exact_match",
    "recall",
    "precision",
    "f1_score",
    "accuracy",
]

overall_metrics = ["exact_match", "recall", "precision", "f1_score", "gpt_score"]


all_metrics = (
    [f"{x}_open" for x in open_metrics]
    + [f"{x}_closed" for x in closed_metrics]
    + [f"{x}_overall" for x in overall_metrics]
)

In [15]:
import tqdm
import json
import numpy as np

bootstrap_indices = {}

seed = 42
rng = np.random.default_rng(seed)
n_bootstraps = 1000

skip_list = ["PathVQA/o3/eval_seed0/none"]

for i in tqdm.tqdm(range(len(df_index))):
    item = df_index.iloc[i]

    dataset = item["dataset"]
    path = item["path"]
    if any([x in path for x in skip_list]):
        continue
    each_example_file = os.path.join(exp_root, path, "each_example_result.csv")
    assert os.path.exists(each_example_file)
    save_file = os.path.join(exp_root, path, "bootstrap_result.csv")

    df_each_example = pd.read_csv(each_example_file)
    bootstrap_results = []

    # generate boot indices for each dataset
    if dataset not in bootstrap_indices.keys():
        indices = []
        for _ in range(n_bootstraps):
            indices.append(rng.integers(0, len(df_each_example), len(df_each_example)))
        bootstrap_indices[dataset] = indices

    for _ in range(n_bootstraps):
        sample_idx = bootstrap_indices[dataset][_]

        df_boot_i = df_each_example.iloc[sample_idx]

        # metrics = {}
        # for m_ in all_metrics:
        #     metrics[m_] = []

        # for idx in sample_idx:
        #     row = df_each_example.iloc[idx]
        #     question_type = row["question_type"]

        #     for m_ in overall_metrics:
        #         metrics[f"{m_}_overall"].append(row[f"{m_}_overall"])

        #     if question_type == "open":
        #         for m_ in open_metrics:
        #             metrics[f"{m_}_open"].append(row[f"{m_}_open"])

        #     if question_type == "closed":
        #         for m_ in closed_metrics:
        #             metrics[f"{m_}_closed"].append(row[f"{m_}_closed"])

        # for k, v in metrics.items():
        #     metrics[k] = np.mean(v)

        bootstrap_results.append(df_boot_i[all_metrics].mean())
    df_boot = pd.DataFrame(bootstrap_results)
    df_boot.to_csv(save_file, index=False)

100%|██████████| 88/88 [02:15<00:00,  1.54s/it]


# generate boot version of results

In [16]:
df_result = pd.read_csv(os.path.join(exp_root, "results.csv"))

new_results = []
for i in range(len(df_result)):
    item = df_result.iloc[i]

    boot_result_i = pd.read_csv(os.path.join(exp_root, item["path"], "bootstrap_result.csv"))

    for m_ in all_metrics:
        item[m_] = boot_result_i[m_].mean()

    new_results.append(item)

df_result_new = pd.DataFrame(new_results)
df_result_new.to_csv(os.path.join(exp_root, "results_bootstrap.csv"), index=False)

/tmp/ipykernel_612887/446734744.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item[m_] = boot_result_i[m_].mean()
/tmp/ipykernel_612887/446734744.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item[m_] = boot_result_i[m_].mean()
/tmp/ipykernel_612887/446734744.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item[m_] = boot_result_i[m_].mean()
/tmp/ipykernel_612887/446734744.py:10: SettingWithCopyWarning: 
A value is t